# Predicting "protein-protein interactions" with neural networks

# Part 2: building the network

This is the second part of the post. For part one, please refer to 
[this post](/post/posts/protein_protein_interaction/).

After we have all the datasets sorted out, the next step in to build the
network. The design of the network is really simple. I am using one CNN to
process one sequence, and then concatenate the two networks together and output
the classification results with a DNN.

### Codes

In [ ]:
from copy import deepcopy

import torch

To make sure we run the model on the right device, we first determine the best
device to use for different platforms.

In [ ]:
if torch.cuda.is_available():
    # with Nvidia GPU
    device = "cuda"
elif torch.has_mps:
    # on Apple Silicon
    device = "mps"
else:
    # on CPU
    device = "cpu"
device = torch.device(device)

Next, let us build the network. I will use 1D CCN here, use the sequences as the
channels. I do not know why, but in my test, this works better than using amino
acid types as channels for 1D CNN or using the sequence and amino acid types for
2D CNN.

First, I will encode the whole sequence with a one-hot encoder.

In [ ]:
class PPIEncoder(torch.nn.Module):
    def __init__(self, one_hot_classes):
        self.one_hot_classes = one_hot_classes
        super().__init__()

    def forward(self, x: torch.tensor):
        return torch.nn.functional.one_hot(
            x.long(), num_classes=self.one_hot_classes
        ).type(torch.float)

Then we will build the actual network.

In [ ]:
class PPICNN(torch.nn.Module):
    def __init__(self, one_hot_classes, output_classes, input_channels=1):
        super().__init__()
        self.cnn1 = torch.nn.Sequential(
            PPIEncoder(one_hot_classes),
            # 1st CNN layer
            torch.nn.Conv1d(
                in_channels=input_channels, out_channels=8, kernel_size=7
            ),
            torch.nn.BatchNorm1d(8),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            # 2st CNN layer
            torch.nn.Conv1d(in_channels=8, out_channels=10, kernel_size=3),
            torch.nn.BatchNorm1d(10),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.MaxPool1d(kernel_size=2, stride=2),
            # 3rd CNN layer
            torch.nn.Conv1d(in_channels=10, out_channels=3, kernel_size=3),
            torch.nn.BatchNorm1d(3),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            # 4th CNN layer
            torch.nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3),
            torch.nn.BatchNorm1d(3),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.MaxPool1d(kernel_size=2, stride=2),
        )

        self.cnn2 = deepcopy(self.cnn1)

        self.fc = torch.nn.Sequential(
            torch.nn.Linear(2316, 80),
            torch.nn.ReLU(),
            torch.nn.Dropout(),
            torch.nn.Linear(80, 40),
            torch.nn.ReLU(),
            torch.nn.Dropout(),
            torch.nn.Linear(40, output_classes),
            torch.nn.Sigmoid(),
        )


    def forward(self, protein1: torch.Tensor, protein2: torch.Tensor):
        protein1 = self.cnn1(protein1)
        protein2 = self.cnn2(protein2)
        combined = torch.cat((protein1, protein2), dim=1).flatten(1)
        return self.fc(combined)

We will process each protein sequence with a 4-layer 1D CNN. Batch
normalization, pooling, dropout, and activation functions are all hyperparameter
you can play with. We set up `cnn1` and get an identical copy for the other
sequence call `cnn2`. Then we concatenate the two networks together and flatten
the resulting tensor to 1D and feed it into a 3-layer fully-connected network.
Sine this is a binary classification problem, we use a `Sigmoid` function at the
end.

Fairly simple, right? In fact, with such a simple set up, we can already achieve
98%+ accuracy. In the next chapter, I will show how I trained this network.